In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
import joblib

In [ ]:
data = pd.read_csv('adjusted_parking_data.csv')

In [ ]:
X = data.drop(columns=['Parking_Slot_Empty'])
y = data['Parking_Slot_Empty']

In [ ]:
X['Hour'] = X['Time_of_Day'].str.split(':').str[0].astype(int)
categorical_features = ['Day_of_Week', 'Season', 'Event']
numerical_features = ['Hour']

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', max_iter=500, random_state=42))
])

In [ ]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Hour']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Day_of_Week', 'Season',
                                                   'Event'])])),
                ('regressor',
                 MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500,
                              random_state=42))])

In [ ]:
y_pred = model_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.14746780416735372


In [ ]:
joblib.dump(model_pipeline, 'parking_lot_regression_model.pkl')

['parking_lot_regression_model.pkl']

In [ ]:
# Load and predict (example)
loaded_model = joblib.load('parking_lot_regression_model.pkl')
example_input = pd.DataFrame({
    'Time_of_Day': ['8:30'],
    'Day_of_Week': ['Monday'],
    'Season': ['Winter'],
    'Event': ['No Event']
})

example_input['Hour'] = example_input['Time_of_Day'].str.split(':').str[0].astype(int)
example_input = example_input.drop(columns=['Time_of_Day'])
prediction = loaded_model.predict(example_input)
print(f"Predicted Probability: {prediction[0]}")

Predicted Probability: 0.6134435290985103
